# Postcode Information

There is a wealth on information available through public Web APIs.  
This notebook explores what you can get with just a postcode, from 5 different APIs.

## API

Application programming interface. This interface defines a prescribed way in which one or more softwares communicate, and has standards around how to make, send, and recieve information (data formats, structure, conventions... etc).

Web APIs more specifically define how URLs (HTTP request messages) can generate a structured response message (often XML or JSON). [For example](https://api.nasa.gov/), you might send your favourite satellite name in a URL and recieve text back containing the current location of its orbit.

### Modules and Postcode

Before doing any coding, three required modules are imported.  
[requests](https://requests.readthedocs.io/en/master/) has the functionality to send HTTP requests and will enable us to make API calls.  
[pandas](https://pypi.org/project/pandas/) enables us
[numpy](https://numpy.org/) 

In [2]:
import requests
import pandas as pd
import numpy as np

The following variable can be changed by a you to vary the outputs in the rest of the code.  
Try putting in a UK postcode of interest to you.

In [3]:
postcode = "CO3 3ND"

### 1) Area Names and Codes

There and many different types of area groupings and cetegories in the UK.  
The [Postcodes.io API](https://postcodes.io/) provides a selection of these for a given postcode.

In [32]:
# Use a get request as we are appending the postcode to the base URL
response1 = requests.get("http://api.postcodes.io/postcodes/"+postcode)

# The status code can shows if the API call was successful, with a value of 200) 
# Other values indicate an error and different numbers are for different issues
print("The status code is "+str(response1.status_code))

# Isolate the location information
location_details = pd.read_json(response1.text)
codes = location_details['result'][5]

# Print the list of location information
print(list(location_details.index))
# Print the "codes" details as well
print(codes)

The status code is 200
['admin_county', 'admin_district', 'admin_ward', 'ccg', 'ced', 'codes', 'country', 'eastings', 'european_electoral_region', 'incode', 'latitude', 'longitude', 'lsoa', 'msoa', 'nhs_ha', 'northings', 'nuts', 'outcode', 'parish', 'parliamentary_constituency', 'postcode', 'primary_care_trust', 'quality', 'region']
                status                      result
admin_county       200                       Essex
admin_district     200                  Colchester
admin_ward         200  New Town and Christ Church
{'admin_district': 'E07000071', 'admin_county': 'E10000012', 'admin_ward': 'E05010835', 'parish': 'E43000228', 'parliamentary_constituency': 'E14000644', 'ccg': 'E38000117', 'ccg_id': '06T', 'ced': 'E58000428', 'nuts': 'UKH34'}


There are a few pieces of location information that will be used in other API calls.  
These are saved to variable below.

In [5]:
pc_lat = float(location_details['result'][['latitude']])
pc_long = float(location_details['result'][['longitude']])
pc_ward = str(codes['admin_ward'])

# Output values
pc_lat, pc_long, pc_ward

(51.886685, 0.886988, 'E05010835')

### 2) Police - Street Crime

The [data.police](https://data.police.uk/docs/) gives access to some police data, including location of street crime.  
Using this, we can seee details of all crimes committed within 1 mile of the postcode (lat/long centroid).  
This request will use a **post** method, as the parameters supplied will appear in the message body of the URL.  
A post request might look like *http://site.com/api/search?data1+data2* in a URL, notice the "?" and what follows.

In [6]:
# The API can take a month of interest in as a parameter, so feel free to edit the below
year_month = "2020-04"

# First, parameters defined earlier in the natebook are stored in a dictionaty
parameters2 = {"date": year_month, "lat": pc_lat, "lng": pc_long}

# These parameters and base URL are submitted as a post API request
response2 = requests.post("https://data.police.uk/api/crimes-street/all-crime", params=parameters2)

# Check the status code is 200
print("The status code is "+str(response2.status_code))

# Extract the crime information into a data frame
crime_df = pd.read_json(response2.text)

# For each crime we get the following information
list(crime_df.columns)

The status code is 200


['category',
 'context',
 'id',
 'location',
 'location_subtype',
 'location_type',
 'month',
 'outcome_status',
 'persistent_id']

In [7]:
print("There were {num} crimes in {month} within a mile of {postcode}.".format(num=len(crime_df.index), 
                                                                               month=year_month, 
                                                                               postcode=postcode))

# These crimes can be grouped by category
crime_df.groupby('category')['category'].count()

There were 391 crimes in 2020-04 within a mile of CO3 3ND.


category
anti-social-behaviour    140
bicycle-theft              7
burglary                   6
criminal-damage-arson     27
drugs                     33
other-crime                9
other-theft               12
public-order              32
robbery                    2
shoplifting                8
vehicle-crime              2
violent-crime            113
Name: category, dtype: int64

### 3) Food Standards Agency
The FSA has [an API](https://api.ratings.food.gov.uk/help) where you can access rating information for eating establishments in your area.  
This API is versioned, and multiple versions can exist at the same time. This requires us to pass a headers input into the API call, where we have defined the correct API version to use.

In [19]:
# The below parameters defines how wide the search area is and how many results to show, you can edit these
# You can edit these, but becareful to pick a large enough search areas to find that many results
miles = 10
obs = 5

# As well as the parameter, we define the API version we want to use
parameters3 = {"latitude": pc_lat, "longitude": pc_long, "maxDistanceLimit": miles, "pageSize": obs}
hdrs = {'x-api-version': '2'}

# Submit the API request, with a params input
response3 = requests.get("https://api.ratings.food.gov.uk/Establishments", params=parameters3, headers=hdrs)

# Check the request was successful
print(response3.status_code)

# Extract the text from the response and get a dataframe containing the establishment information
text = pd.read_json("["+response3.text+"]")
establisments = pd.DataFrame.from_dict(text['establishments'][0])

# These are the details we recieved on each establishment
print(establisments.columns)

200
Index(['AddressLine1', 'AddressLine2', 'AddressLine3', 'AddressLine4',
       'BusinessName', 'BusinessType', 'BusinessTypeID', 'Distance', 'FHRSID',
       'LocalAuthorityBusinessID', 'LocalAuthorityCode',
       'LocalAuthorityEmailAddress', 'LocalAuthorityName',
       'LocalAuthorityWebSite', 'NewRatingPending', 'Phone', 'PostCode',
       'RatingDate', 'RatingKey', 'RatingValue', 'RightToReply', 'SchemeType',
       'geocode', 'links', 'meta', 'scores'],
      dtype='object')


The below loop will go through all the establishments and print some key information on them.  
This loop will be as long as the **obs** value defined above.

In [21]:
# This loop iterates through all establishements and prints the same start but with the information specific to each
for i in range(len(establisments.index)):
    print("{est} at {pc} is {dist} miles away, and had a rating value of {rc} on {date}.".format(
        est = establisments['BusinessName'][i],
        pc = establisments['PostCode'][i],
        dist = round(establisments['Distance'][i],2),
        rc = establisments['RatingValue'][i],
        date = establisments['RatingDate'][i])
         )

222 Coffee Shop at CO1 1LH is 0.67 miles away, and had a rating value of 5 on 2020-02-07T00:00:00.
5FiVE Bars Limited at CO4 9QQ is 2.97 miles away, and had a rating value of 5 on 2019-10-09T00:00:00.
A B Hotels Limited at CM9 8HX is 7.75 miles away, and had a rating value of 5 on 2018-11-13T00:00:00.
A Leeder Butchers at CO10 5NZ is 9.94 miles away, and had a rating value of 5 on 2019-07-08T00:00:00.
A Willsher and Son at CO6 1EB is 4.4 miles away, and had a rating value of 5 on 2018-11-23T00:00:00.


### 4) Public Health England
Public health have a store of information called fingertips which has a [web API](https://fingertips.phe.org.uk/api).  
Not only do they have API, but there is a [python package](https://fingertips-py.readthedocs.io/en/latest/) which simplifies the process of interacting with the API. 

In [25]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install fingertips_py

import fingertips_py as ftp

  Stored in directory: C:\Users\Henry\AppData\Local\pip\Cache\wheels\ad\20\e8\3350b611979d0f7a87d8505078302dafc040ef2e718d5cf19f
Successfully built fingertips-py


In [30]:
healthy_life_data = ftp.get_data_for_indicator_at_all_available_geographies(90362)
healthy_life_data.head(3)

pandas.core.frame.DataFrame